<a href="https://colab.research.google.com/github/Milechwan/if1015/blob/master/Projeto2_IF1015.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import copy

dataset = pd.read_csv('https://raw.githubusercontent.com/Milechwan/if1015/master/googleplaystore.csv',encoding = "ISO-8859-1")

In [2]:
dataset.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,Art & Design,"January 7, 2018",1.0.0,4.0.3 and up
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,Art & Design;Pretend Play,"January 15, 2018",2.0.0,4.0.3 and up
2,"U Launcher Lite â FREE Live Cool Themes, Hid...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,Art & Design,"August 1, 2018",1.2.4,4.0.3 and up
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,Art & Design,"June 8, 2018",Varies with device,4.2 and up
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,Art & Design;Creativity,"June 20, 2018",1.1,4.4 and up


Objetivo: Descobrir qual o melhor modelo para predizer a nota de avaliação (rating) dos 
aplicativos

In [3]:
dataset[dataset['Rating'].isna()]

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,Last Updated,Current Ver,Android Ver
23,Mcqueen Coloring pages,ART_AND_DESIGN,NaN,61,7.0M,"100,000+",Free,0,Everyone,Art & Design;Action & Adventure,"March 7, 2018",1.0.0,4.1 and up
113,Wrinkles and rejuvenation,BEAUTY,NaN,182,5.7M,"100,000+",Free,0,Everyone 10+,Beauty,"September 20, 2017",8.0,3.0 and up
123,Manicure - nail design,BEAUTY,NaN,119,3.7M,"50,000+",Free,0,Everyone,Beauty,"July 23, 2018",1.3,4.1 and up
126,Skin Care and Natural Beauty,BEAUTY,NaN,654,7.4M,"100,000+",Free,0,Teen,Beauty,"July 17, 2018",1.15,4.1 and up
129,"Secrets of beauty, youth and health",BEAUTY,NaN,77,2.9M,"10,000+",Free,0,Mature 17+,Beauty,"August 8, 2017",2.0,2.3 and up
130,Recipes and tips for losing weight,BEAUTY,NaN,35,3.1M,"10,000+",Free,0,Everyone 10+,Beauty,"December 11, 2017",2.0,3.0 and up
134,"Lady adviser (beauty, health)",BEAUTY,NaN,30,9.9M,"10,000+",Free,0,Mature 17+,Beauty,"January 24, 2018",3.0,3.0 and up
163,Anonymous caller detection,BOOKS_AND_REFERENCE,NaN,161,2.7M,"10,000+",Free,0,Everyone,Books & Reference,"July 13, 2018",1.0,2.3 and up
180,SH-02J Owner's Manual (Android 8.0),BOOKS_AND_REFERENCE,NaN,2,7.2M,"50,000+",Free,0,Everyone,Books & Reference,"June 15, 2018",3.0,6.0 and up
185,URBANO V 02 instruction manual,BOOKS_AND_REFERENCE,NaN,114,7.3M,"100,000+",Free,0,Everyone,Books & Reference,"August 7, 2015",1.1,5.1 and up


Primeiramente, deve-se remover os dados faltantes e dados duplicados (comparando pelo nome)

In [4]:
dataset = dataset.dropna(subset=['Rating'])
dataset.drop_duplicates(subset=['App'],keep='last', inplace=True)
print(len(dataset))

8197


Transformação de colunas categóricas (definidas no Projeto 1), e remoção do símbolo $ da coluna Price para transformar em tipo numérico. Adicionalmente, é removida a linha (conforme foi também visto no Projeto 1) que possui os valores nas colunas incorretas, para transformar a coluna Reviews em tipo numérico.

In [0]:
dataset = dataset.drop(dataset[dataset['Price']=='Everyone'].index, axis = 0)
dataset['Reviews'] = pd.to_numeric(dataset['Reviews'])

In [0]:
dataset['Price'] = dataset['Price'].apply(lambda x: x[x.find('$')+1:] if x.find('$')>-1 else x)
dataset['Price'] = pd.to_numeric(dataset['Price'])

In [0]:
dataset['Last Updated'] = pd.to_datetime(dataset['Last Updated'])
dataset['Last Updated'] = dataset['Last Updated'].apply(lambda date: date.timestamp())#colocando em timestamp (milissegundos) para que fique como valor numérico e possa ser passado no conjunto de variáveis X adiante 

In [0]:
dataset['Type'] = dataset['Type'].astype("category")
dataset['Content Rating'] = dataset['Content Rating'].astype("category") 
dataset['Category'] = dataset['Category'].astype("category") 
dataset['Installs'] = dataset['Installs'].astype("category")
dataset['Android Ver'] = dataset['Android Ver'].astype("category")
dataset['Current Ver'] = dataset['Current Ver'].astype("category")
dataset['Size'] = dataset['Size'].astype("category")

dataset['Type'] = dataset['Type'].cat.codes
dataset['Content Rating'] = dataset['Content Rating'].cat.codes
dataset['Category'] = dataset['Category'].cat.codes
dataset['Installs'] = dataset['Installs'].cat.codes
dataset['Android Ver'] = dataset['Android Ver'].cat.codes
dataset['Current Ver'] = dataset['Current Ver'].cat.codes
dataset['Size'] = dataset['Size'].cat.codes

In [9]:
dataset.corr(method='spearman')

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Last Updated,Current Ver,Android Ver
Category,1.000000,-0.034912,0.054965,-0.013080,0.004125,0.020257,0.019350,-0.122684,0.023375,0.112800,0.092330
Rating,-0.034912,1.000000,0.119318,0.041672,0.000542,0.071833,0.070594,-0.005499,0.181590,0.005516,0.070131
Reviews,0.054965,0.119318,1.000000,0.290101,0.116169,-0.171393,-0.170257,0.172131,0.373522,0.393281,0.218088
Size,-0.013080,0.041672,0.290101,1.000000,0.044403,-0.029253,-0.029454,0.068271,0.187546,0.317859,0.322931
Installs,0.004125,0.000542,0.116169,0.044403,1.000000,-0.044453,-0.044741,0.031409,0.039878,0.037882,0.025105
Type,0.020257,0.071833,-0.171393,-0.029253,-0.044453,1.000000,0.999045,-0.031581,-0.148605,-0.065439,-0.089315
Price,0.019350,0.070594,-0.170257,-0.029454,-0.044741,0.999045,1.000000,-0.030736,-0.148130,-0.064642,-0.088425
Content Rating,-0.122684,-0.005499,0.172131,0.068271,0.031409,-0.031581,-0.030736,1.000000,0.085278,-0.008345,-0.005885
Last Updated,0.023375,0.181590,0.373522,0.187546,0.039878,-0.148605,-0.148130,0.085278,1.000000,0.347126,0.477804
Current Ver,0.112800,0.005516,0.393281,0.317859,0.037882,-0.065439,-0.064642,-0.008345,0.347126,1.000000,0.408912


Remoção de outliers com DBSCAN

In [11]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

#Normalizando dataset para que o DBSCAN funcione corretamente para calcular as distâncias e clusterizar os dados
#removendo colunas inutilizadas 
dataset = dataset.drop('Genres', axis = 1)
dataset = dataset.drop('App', axis = 1)

normalized_ds = copy.deepcopy(dataset)
normalized_ds = scaler.fit_transform(normalized_ds)
normalized_ds = pd.DataFrame(normalized_ds,columns=["Category","Rating","Reviews","Size","Installs","Type","Price","Content Rating","Last Updated","Current Ver","Android Ver"])
normalized_ds.head()

,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Last Updated,Current Ver,Android Ver
0,0.0,0.775,0.000002,0.116223,0.277778,0.0,0.0,0.2,0.929024,0.041143,0.483871
1,0.0,0.925,0.001120,0.808717,0.666667,0.0,0.0,0.2,0.997667,0.161143,0.483871
2,0.0,0.875,0.002760,0.213075,0.833333,0.0,0.0,0.8,0.979673,0.979048,0.580645
3,0.0,0.825,0.000012,0.138015,0.444444,0.0,0.0,0.2,0.983672,0.095619,0.645161
4,0.0,0.850,0.000002,0.489104,0.777778,0.0,0.0,0.2,0.833389,0.039619,0.258065


In [0]:
from sklearn.cluster import DBSCAN
outlier_detection = DBSCAN(
  eps = 0.3,
  metric="euclidean",
  min_samples = 10)
clusters = outlier_detection.fit_predict(normalized_ds)

dataset_without_outliers = copy.deepcopy(dataset) #criando cópia para comparar resultado dos modelos com e sem os outliers 

In [13]:
#labels guardam os valores que identificam se determinado item do dataset é ou não um outlier.
labels = outlier_detection.labels_

#verificação do número de grupos (clusters) identificados pelo DBSCAN 
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
print("Total clusters: "+str(n_clusters_))
n_noise_ = list(labels).count(-1)#outliers 
print("Total outliers: "+str(n_noise_))

Total clusters: 5
Total outliers: 902


In [14]:
#identificando os índices dos itens que são outliers para removê-los do conjunto de dados
index_outlier = np.where(labels == -1)
result = index_outlier[0]
print(result.shape)

(902,)


In [15]:
dataset_without_outliers.drop(dataset_without_outliers.index[result], inplace = True)
print(len(dataset))
print(len(dataset_without_outliers))

8196
7294



Modelo de predição Linear Regression (sem remover outliers)

In [0]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,recall_score,precision_score,f1_score

model = LinearRegression()

dataset_without_rating = copy.deepcopy(dataset)
dataset_without_rating = dataset_without_rating.drop('Rating',axis=1)
X = np.array(dataset_without_rating)
y = np.array(dataset['Rating'])
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.33,
                                                random_state=42)

In [17]:
model.fit(Xtrain,ytrain)
y_model = model.predict(Xtest)
y_pred_train = model.predict(Xtrain)
print(model.score(Xtest,ytest))
print('MAPE: '+str(np.mean(np.abs((ytest-y_model)/ytest))*100))

0.022322088429774792
MAPE: 10.249329143592947


In [18]:
from sklearn.metrics import explained_variance_score, max_error, mean_absolute_error, mean_squared_log_error,make_scorer

print('Explained variance score: '+ str(explained_variance_score(ytest,y_model)))
print('Max error: '+ str(max_error(ytest,y_model)))
print('Mean absolute error: '+str(mean_absolute_error(ytest,y_model)))
print('Mean squared log error: '+str(mean_squared_log_error(ytest,y_model)))

Explained variance score: 0.022753992329419437
Max error: 3.2644439385063233
Mean absolute error: 0.36809210530415876
Mean squared log error: 0.01282204559081877


Linear Regression (com remoção de outliers)

In [19]:
without_outliers_copy = copy.deepcopy(dataset_without_outliers)
X_without_outliers = np.array(without_outliers_copy.drop('Rating',axis=1))
y_without_outliers = np.array(without_outliers_copy['Rating'])
X_without_out_train, X_without_out_test, y_without_out_train, y_without_out_test = train_test_split(X, y, test_size=0.33,
                                                random_state=42)

model_without_outliers = LinearRegression()
model_without_outliers.fit(X_without_out_train,y_without_out_train)
y_model_without_outliers = model_without_outliers.predict(X_without_out_test)
print(model_without_outliers.score(X_without_out_test,y_without_out_test))

0.022322088429774792


In [20]:
print('Explained variance score: '+ str(explained_variance_score(y_without_out_test,y_model_without_outliers)))
print('Max error: '+ str(max_error(y_without_out_test,y_model_without_outliers)))
print('Mean absolute error: '+str(mean_absolute_error(y_without_out_test,y_model_without_outliers)))
print('Mean squared log error: '+str(mean_squared_log_error(y_without_out_test,y_model_without_outliers)))
print('MAPE: '+str(np.mean(np.abs((y_without_out_test-y_model_without_outliers)/y_without_out_test))*100))

Explained variance score: 0.022753992329419437
Max error: 3.2644439385063233
Mean absolute error: 0.36809210530415876
Mean squared log error: 0.01282204559081877
MAPE: 10.249329143592947


Usando SelectFromModel, que dado o estimador e as variáveis X e y, ele retorna um array contendo toda a base mas selecionando as variáveis independentes mais adequadas para fazer a predição. Método de redução de dimensionalidade

In [0]:
from sklearn.feature_selection import SelectFromModel
feature = SelectFromModel(model)#sem remover outliers
Fit = feature.fit_transform(X, y)

In [22]:
print(Fit.shape)

(8196, 1)


SelectFromModel sem remoção de outliers

In [23]:
Fittrain, Fittest, yfittrain, yfittest = train_test_split(Fit, y, test_size=0.33,
                                                random_state=42)
model.fit(Fittrain,yfittrain)
y_model_fit = model.predict(Fittest)
print(model.score(Fittest,yfittest))

0.002142380562441204


In [24]:
print('MAPE: '+str(np.mean(np.abs((yfittest-y_model_fit)/yfittest))*100))
print('Explained variance score: '+ str(explained_variance_score(yfittest,y_model_fit)))
print('Max error: '+ str(max_error(yfittest,y_model_fit)))
print('Mean absolute error: '+str(mean_absolute_error(yfittest,y_model_fit)))
print('Mean squared log error: '+str(mean_squared_log_error(yfittest,y_model_fit)))

MAPE: 10.447092863344764
Explained variance score: 0.0027433949763074894
Max error: 3.1623034591194967
Mean absolute error: 0.37618109848603193
Mean squared log error: 0.013036935206045213


RandomForest

In [0]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators = 100, random_state = 42)

Random Forest sem remoção de outliers

In [26]:
rf.fit(Xtrain,ytrain)

y_rf_model = rf.predict(Xtest)

print('R² score: '+ str(rf.score(Xtest,ytest)))
print('MAPE: '+str(np.mean(np.abs((ytest-y_rf_model)/ytest))*100))
print('Explained variance score: '+ str(explained_variance_score(ytest,y_rf_model)))
print('Max error: '+ str(max_error(ytest,y_rf_model)))
print('Mean absolute error: '+str(mean_absolute_error(ytest,y_rf_model)))
print('Mean squared log error: '+str(mean_squared_log_error(ytest,y_rf_model)))

R² score: 0.09324801914095426
MAPE: 9.71207803411096
Explained variance score: 0.09815102978401025
Max error: 3.139999999999999
Mean absolute error: 0.35344325323475045
Mean squared log error: 0.011871899692637326


Random Forest com remoção de outliers

In [68]:
rf = RandomForestRegressor(n_estimators = 100, random_state = 42)

rf.fit(X_without_out_train,y_without_out_train)
result_rf_without_outliers = rf.predict(X_without_out_test)
print('R² score: '+str(rf.score(X_without_out_test,y_without_out_test)))
print('MAPE: '+str(np.mean(np.abs((y_without_out_test-result_rf_without_outliers)/y_without_out_test))*100))
print('Explained variance score: '+ str(explained_variance_score(y_without_out_test,result_rf_without_outliers)))
print('Max error: '+ str(max_error(y_without_out_test,result_rf_without_outliers)))
print('Mean absolute error: '+str(mean_absolute_error(y_without_out_test,result_rf_without_outliers)))
print('Mean squared log error: '+str(mean_squared_log_error(y_without_out_test,result_rf_without_outliers)))

R² score: 0.09324801914095426
MAPE: 9.71207803411096
Explained variance score: 0.09815102978401025
Max error: 3.139999999999999
Mean absolute error: 0.35344325323475045
Mean squared log error: 0.011871899692637326


Support Vector Regression - outro modelo de predição apenas para comparar desempenho e ver qual modelo de fato vale a pena tentar melhorar 

In [85]:
from sklearn.svm import SVR

svr_rbf = SVR(kernel='rbf', gamma=0.1, epsilon=.1)

svr_rbf.fit(Xtrain,ytrain)
result_svr_rbf = svr_rbf.predict(Xtest)

print('R² score: '+ str(svr_rbf.score(Xtest,ytest)))
print('MAPE: '+str(np.mean(np.abs((ytest-result_svr_rbf)/ytest))*100))
print('Explained variance score: '+ str(explained_variance_score(ytest,result_svr_rbf)))
print('Max error: '+ str(max_error(ytest,result_svr_rbf)))
print('Mean absolute error: '+str(mean_absolute_error(ytest,result_svr_rbf)))
print('Mean squared log error: '+str(mean_squared_log_error(ytest,result_svr_rbf)))

R² score: -0.002261046000373357
MAPE: 10.447759118627772
Explained variance score: -0.0021005645805545203
Max error: 3.1746894398570005
Mean absolute error: 0.3756397111297119
Mean squared log error: 0.013088781130181014


SVR no dataset sem outliers

In [29]:
svr_rbf = SVR(kernel='rbf', gamma=0.1, epsilon=.1)

svr_rbf.fit(X_without_out_train,y_without_out_train)
result_svr_rbf = svr_rbf.predict(X_without_out_test)
print('R² score: '+str(svr_rbf.score(X_without_out_test,y_without_out_test)))
print('MAPE: '+str(np.mean(np.abs((y_without_out_test-result_svr_rbf)/y_without_out_test))*100))
print('Explained variance score: '+ str(explained_variance_score(y_without_out_test,result_svr_rbf)))
print('Max error: '+ str(max_error(y_without_out_test,result_svr_rbf)))
print('Mean absolute error: '+str(mean_absolute_error(y_without_out_test,result_svr_rbf)))
print('Mean squared log error: '+str(mean_squared_log_error(y_without_out_test,result_svr_rbf)))

R² score: -0.002261046000373357
MAPE: 10.447759118627772
Explained variance score: -0.0021005645805545203
Max error: 3.1746894398570005
Mean absolute error: 0.3756397111297119
Mean squared log error: 0.013088781130181014


In [30]:
svr_sigmoid = SVR(kernel='sigmoid', epsilon=.33)

svr_sigmoid.fit(Xtrain,ytrain)
result_svr_sigmoid = svr_sigmoid.predict(Xtest)
print('R² score: '+str(svr_sigmoid.score(Xtest,ytest)))
print('MAPE: '+str(np.mean(np.abs((ytest-result_svr_sigmoid)/ytest))*100))
print('Explained variance score: '+ str(explained_variance_score(ytest,result_svr_sigmoid)))
print('Max error: '+ str(max_error(ytest,result_svr_sigmoid)))
print('Mean absolute error: '+str(mean_absolute_error(ytest,result_svr_sigmoid)))
#print('Mean squared log error: '+str(mean_squared_log_error(ytest,result_svr_sigmoid)))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


R² score: -0.008505067534084088
MAPE: 10.36309772411867
Explained variance score: 0.0
Max error: 3.2299999999999995
Mean absolute error: 0.36632532347504626


PassiveAggressiveRegressor - sem remoção de outliers

In [62]:
from sklearn.linear_model import PassiveAggressiveRegressor
from sklearn import preprocessing
from sklearn import utils
from sklearn.preprocessing import StandardScaler

model_pa = PassiveAggressiveRegressor(random_state=42)

pa_X_train, pa_X_test, pa_y_train, pa_y_test = train_test_split(X, y, test_size=0.33, random_state=42)

model_pa.fit(pa_X_train,pa_y_train)
y_lr = model_lr.predict(standardized_X_test)
print('R² score: '+str(model_pa.score(pa_X_test,pa_y_test)))
print('MAPE: '+str(np.mean(np.abs((pa_y_test-y_lr)/pa_y_test))*100))
print('Explained variance score: '+ str(explained_variance_score(pa_y_test,y_lr)))
print('Max error: '+ str(max_error(pa_y_test,y_lr)))
print('Mean absolute error: '+str(mean_absolute_error(pa_y_test,y_lr)))

R² score: -0.08528730970856802
MAPE: 10.534096542343862
Explained variance score: 0.003049480263088711
Max error: 3.401197079000048
Mean absolute error: 0.3628870309698339


# Grid Search (com cross validation)



1.   Random Forest Regressor



In [67]:
from sklearn.model_selection import GridSearchCV

metrics = ['explained_variance','neg_mean_absolute_error','neg_mean_squared_log_error','max_error','r2']
rf.get_params().keys()

dict_keys(['C', 'average', 'early_stopping', 'epsilon', 'fit_intercept', 'loss', 'max_iter', 'n_iter_no_change', 'random_state', 'shuffle', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

In [76]:
dict_ = {'n_estimators': [100,80,50], 'random_state': [42]}
dict_less_estimators = {'n_estimators': [80], 'random_state': [42]}

rf = RandomForestRegressor()

first_grid = GridSearchCV(rf, param_grid=dict_, scoring=make_scorer(mean_absolute_error), verbose=100, refit='neg_mean_absolute_error', return_train_score=False)

first_grid.fit(Xtrain,ytrain)


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


Fitting 3 folds for each of 3 candidates, totalling 9 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] n_estimators=100, random_state=42 ...............................
[CV] ... n_estimators=100, random_state=42, score=0.358, total=   1.7s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.7s remaining:    0.0s
[CV] n_estimators=100, random_state=42 ...............................
[CV] ... n_estimators=100, random_state=42, score=0.357, total=   1.6s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    3.3s remaining:    0.0s
[CV] n_estimators=100, random_state=42 ...............................
[CV] ... n_estimators=100, random_state=42, score=0.361, total=   1.6s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    5.0s remaining:    0.0s
[CV] n_estimators=80, random_state=42 ................................
[CV] .... n_estimators=80, random_state=42, score=0.357, total=   1.3s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:

GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=RandomForestRegressor(bootstrap=True, criterion='mse',
                                             max_depth=None,
                                             max_features='auto',
                                             max_leaf_nodes=None,
                                             min_impurity_decrease=0.0,
                                             min_impurity_split=None,
                                             min_samples_leaf=1,
                                             min_samples_split=2,
                                             min_weight_fraction_leaf=0.0,
                                             n_estimators='warn', n_jobs=None,
                                             oob_score=False, random_state=None,
                                             verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'n_estimators': [100, 80, 

In [77]:
gridsearch_rf = RandomForestRegressor()
gridsearch_rf.set_params(**first_grid.best_params_)
gridsearch_rf.fit(Xtrain,ytrain)
gs_model = gridsearch_rf.predict(Xtest)
print('R² score: '+str(gridsearch_rf.score(Xtest,ytest)))
print('MAPE: '+str(np.mean(np.abs((ytest-gs_model)/ytest))*100))
print('Explained variance score: '+ str(explained_variance_score(ytest,gs_model)))
print('Max error: '+ str(max_error(ytest,gs_model)))
print('Mean absolute error: '+str(mean_absolute_error(ytest,gs_model)))
print('Mean squared log error: '+str(mean_squared_log_error(ytest,gs_model)))

R² score: 0.08369601511000879
MAPE: 9.752052765713534
Explained variance score: 0.08906393751223629
Max error: 3.1980000000000004
Mean absolute error: 0.35511275415896487
Mean squared log error: 0.01198770563344003


In [78]:
first_grid.best_params_

{'n_estimators': 50, 'random_state': 42}


2.   Linear Regression



In [79]:
linear_regression = LinearRegression()

linear_regression.get_params().keys() 

dict_keys(['copy_X', 'fit_intercept', 'n_jobs', 'normalize'])

In [80]:
dict_lr = {'fit_intercept':[True,False],'normalize':[True,False]}

second_grid = GridSearchCV(linear_regression, param_grid=dict_lr, scoring=make_scorer(mean_absolute_error), verbose=100, refit='neg_mean_absolute_error', return_train_score=False)
second_grid.fit(X,y)

Fitting 3 folds for each of 4 candidates, totalling 12 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] fit_intercept=True, normalize=True ..............................
[CV] .. fit_intercept=True, normalize=True, score=0.310, total=   0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[CV] fit_intercept=True, normalize=True ..............................
[CV] .. fit_intercept=True, normalize=True, score=0.416, total=   0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[CV] fit_intercept=True, normalize=True ..............................
[CV] .. fit_intercept=True, normalize=True, score=0.416, total=   0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[CV] fit_intercept=True, normalize=False .............................
[CV] . fit_intercept=True, normalize=False, score=0.310, total=   0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=LinearRegression(copy_X=True, fit_intercept=True,
                                        n_jobs=None, normalize=False),
             iid='warn', n_jobs=None,
             param_grid={'fit_intercept': [True, False],
                         'normalize': [True, False]},
             pre_dispatch='2*n_jobs', refit='neg_mean_absolute_error',
             return_train_score=False, scoring=make_scorer(mean_absolute_error),
             verbose=100)

In [84]:
lr2 = LinearRegression()
lr2.set_params(**second_grid.best_params_)
lr2.fit(Xtrain,ytrain)
lr2_model = lr2.predict(Xtest)
print('R² score: '+str(lr2.score(Xtest,ytest)))
print('MAPE: '+str(np.mean(np.abs((ytest-lr2_model)/ytest))*100))
print('Explained variance score: '+ str(explained_variance_score(ytest,lr2_model)))
print('Max error: '+ str(max_error(ytest,lr2_model)))
print('Mean absolute error: '+str(mean_absolute_error(ytest,lr2_model)))
print('Mean squared log error: '+str(mean_squared_log_error(ytest,lr2_model)))

R² score: 0.022322088429774792
MAPE: 10.249329143592947
Explained variance score: 0.022753992329419437
Max error: 3.2644439385063233
Mean absolute error: 0.36809210530415876
Mean squared log error: 0.01282204559081877


3. Support Vector Regressor

In [86]:
svr_grid = SVR()
svr_grid.get_params().keys()
#SVR(kernel='sigmoid', epsilon=.33)

dict_keys(['C', 'cache_size', 'coef0', 'degree', 'epsilon', 'gamma', 'kernel', 'max_iter', 'shrinking', 'tol', 'verbose'])

In [88]:
dict_svr={'C':[1,10,50,100],'epsilon':[0.1,0.33,0.5],'kernel':['rbf','sigmoid']}
third_grid = GridSearchCV(svr_grid,param_grid=dict_svr,scoring=make_scorer(mean_absolute_error), verbose=100, refit='neg_mean_absolute_error', return_train_score=False)
third_grid.fit(Xtrain,ytrain)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Fitting 3 folds for each of 24 candidates, totalling 72 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] C=1, epsilon=0.1, kernel=rbf ....................................
[CV] ........ C=1, epsilon=0.1, kernel=rbf, score=0.377, total=   1.4s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s
[CV] C=1, epsilon=0.1, kernel=rbf ....................................


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[CV] ........ C=1, epsilon=0.1, kernel=rbf, score=0.388, total=   1.4s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.8s remaining:    0.0s
[CV] C=1, epsilon=0.1, kernel=rbf ....................................


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


...

[CV] ...... C=100, epsilon=0.5, kernel=rbf, score=0.488, total=   0.5s
[Parallel(n_jobs=1)]: Done  69 out of  69 | elapsed:   46.1s remaining:    0.0s
[CV] C=100, epsilon=0.5, kernel=sigmoid ..............................
[CV] .. C=100, epsilon=0.5, kernel=sigmoid, score=0.396, total=   0.2s
[Parallel(n_jobs=1)]: Done  70 out of  70 | elapsed:   46.3s remaining:    0.0s
[CV] C=100, epsilon=0.5, kernel=sigmoid ..............................


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


[CV] .. C=100, epsilon=0.5, kernel=sigmoid, score=0.407, total=   0.2s
[Parallel(n_jobs=1)]: Done  71 out of  71 | elapsed:   46.5s remaining:    0.0s
[CV] C=100, epsilon=0.5, kernel=sigmoid ..............................
[CV] .. C=100, epsilon=0.5, kernel=sigmoid, score=0.384, total=   0.2s
[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed:   46.7s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed:   46.7s finished


/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [1, 10, 50, 100], 'epsilon': [0.1, 0.33, 0.5],
                         'kernel': ['rbf', 'sigmoid']},
             pre_dispatch='2*n_jobs', refit='neg_mean_absolute_error',
             return_train_score=False, scoring=make_scorer(mean_absolute_error),
             verbose=100)

In [89]:
svr2 = SVR()
svr2.set_params(**third_grid.best_params_)
svr2.fit(Xtrain,ytrain)
svr2_model = lr2.predict(Xtest)
print('R² score: '+str(svr2.score(Xtest,ytest)))
print('MAPE: '+str(np.mean(np.abs((ytest-svr2_model)/ytest))*100))
print('Explained variance score: '+ str(explained_variance_score(ytest,svr2_model)))
print('Max error: '+ str(max_error(ytest,svr2_model)))
print('Mean absolute error: '+str(mean_absolute_error(ytest,svr2_model)))
print('Mean squared log error: '+str(mean_squared_log_error(ytest,svr2_model)))

/usr/local/lib/python3.6/dist-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


R² score: -37.738985110586455
MAPE: 10.249329143592947
Explained variance score: 0.022753992329419437
Max error: 3.2644439385063233
Mean absolute error: 0.36809210530415876
Mean squared log error: 0.01282204559081877


4. Passive Aggressive Regressor

In [90]:
pa_grid = PassiveAggressiveRegressor()
pa_grid.get_params().keys()

dict_keys(['C', 'average', 'early_stopping', 'epsilon', 'fit_intercept', 'loss', 'max_iter', 'n_iter_no_change', 'random_state', 'shuffle', 'tol', 'validation_fraction', 'verbose', 'warm_start'])

In [92]:
dict_pa={'C':[1,10,50,100],'epsilon':[.001,.01,.1],'random_state':[42]}
fourth_grid = GridSearchCV(pa_grid,param_grid=dict_pa,scoring=make_scorer(mean_absolute_error), verbose=100, refit='neg_mean_absolute_error', return_train_score=False)
fourth_grid.fit(Xtrain,ytrain)

Fitting 3 folds for each of 12 candidates, totalling 36 fits
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[CV] C=1, epsilon=0.001, random_state=42 .............................
[CV] . C=1, epsilon=0.001, random_state=42, score=0.366, total=   0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[CV] C=1, epsilon=0.001, random_state=42 .............................
[CV] . C=1, epsilon=0.001, random_state=42, score=0.457, total=   0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[CV] C=1, epsilon=0.001, random_state=42 .............................
[CV] . C=1, epsilon=0.001, random_state=42, score=0.478, total=   0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.0s remaining:    0.0s
[CV] C=1, epsilon=0.01, random_state=42 ..............................
[CV] .. C=1, epsilon=0.01, random_state=42, score=0.365, total=   0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapse

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=PassiveAggressiveRegressor(C=1.0, average=False,
                                                  early_stopping=False,
                                                  epsilon=0.1,
                                                  fit_intercept=True,
                                                  loss='epsilon_insensitive',
                                                  max_iter=1000,
                                                  n_iter_no_change=5,
                                                  random_state=None,
                                                  shuffle=True, tol=0.001,
                                                  validation_fraction=0.1,
                                                  verbose=0, warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [1, 10, 50, 100], 'epsilon': [0.001, 0.01, 0.1],
                         'random_stat

In [93]:
pa2 = PassiveAggressiveRegressor()
pa2.set_params(**fourth_grid.best_params_)
pa2.fit(Xtrain,ytrain)
pa2_model = pa2.predict(Xtest)
print('R² score: '+str(pa2.score(Xtest,ytest)))
print('MAPE: '+str(np.mean(np.abs((ytest-pa2_model)/ytest))*100))
print('Explained variance score: '+ str(explained_variance_score(ytest,pa2_model)))
print('Max error: '+ str(max_error(ytest,pa2_model)))
print('Mean absolute error: '+str(mean_absolute_error(ytest,pa2_model)))
print('Mean squared log error: '+str(mean_squared_log_error(ytest,pa2_model)))

R² score: -0.21427303899847308
MAPE: 11.211615302002272
Explained variance score: 0.002520473426551817
Max error: 3.490376486044104
Mean absolute error: 0.3832513915545792
Mean squared log error: 0.015691014676455884


**Com e sem a busca de hiper-parâmetros, comparando-se os valores de MAPE, o preditor que obteve os menores resultados foi o Random Forest**